In [ ]:
# autoreload
%load_ext autoreload
%autoreload 2

In [ ]:
import os

os.chdir("..")

In [ ]:
import torch
from omegaconf import OmegaConf
import segmentation_models_pytorch as smp
from train_utils import prepare_data, prepare_model, prepare_optimizer, train, set_seeds

In [ ]:
# import numpy as np
# import yaml
# import argparse
# from omegaconf import OmegaConf
# import wandb
# import torch
# import torchvision.transforms.v2 as transforms
# from models.UNetV1 import UNetV1
# from models.UNetV2 import UNetV2
# from models.UNetV3 import UNetV3
# from models.DeepLabV3 import ResNet101
# from datasets.BaseDataset import BaseDataset
# from datasets.TransformDataset import TransformDataset
# import random


# # set seeds as function
# def set_seeds(seed=42):
#     random.seed(seed)
#     np.random.seed(seed)
#     torch.manual_seed(seed)
#     torch.cuda.manual_seed(seed)


# def prepare_transforms(args):
#     # prepare random transform (to be applied at the same time to both image and groundtruth, for consistency)
#     # the goal is to avoid scenarios like flipping the image but not the groundtruth.
#     random_transform = []
#     # random resized crop
#     if args.random_resized_crop:
#         print(
#             f"Using RandomResizedCrop with output size={tuple(args.output_size)}, scale={tuple(args.random_resized_crop_scale)}."
#         )
#         random_transform.append(
#             transforms.RandomResizedCrop(
#                 size=tuple(args.output_size),
#                 # scale=tuple(args.random_resized_crop_scale),
#             )
#         )
#     # resize to 608x608, or pad 416x416
#     # resize_transform = transforms.Pad(padding=8)
#     resize_transform = transforms.Resize((384, 384))

#     # random horizontal flip
#     if args.random_horizontal_flip:
#         print("Using RandomHorizontalFlip.")
#         random_transform.append(transforms.RandomHorizontalFlip())
#     # random vertical flip
#     if args.random_vertical_flip:
#         print("Using RandomVerticalFlip.")
#         random_transform.append(transforms.RandomVerticalFlip())
#     # random rotation
#     if args.random_rotation:
#         print(f"Using RandomRotation with degrees={args.degrees}.")
#         random_transform.append(transforms.RandomRotation(degrees=args.degrees))
#     # prepare additional transforms for image and groundtruth (these do not need to be applied at the same time)
#     # the goal is to allow some flexibility in the transforms applied to the image and groundtruth, e.g. color jitter for image only.
#     image_transform = []
#     gt_transform = []
#     # color jitter (for image only)
#     if args.color_jitter:
#         print(
#             f"Using ColorJitter with brightness={args.brightness}, contrast={args.contrast}, saturation={args.saturation}, hue={args.hue}."
#         )
#         image_transform.append(
#             transforms.ColorJitter(
#                 brightness=args.brightness,
#                 contrast=args.contrast,
#                 saturation=args.saturation,
#                 hue=args.hue,
#             )
#         )
#     # convert to tensors

#     image_transform.append(transforms.ToTensor())
#     gt_transform.append(transforms.ToTensor())
#     image_transform.append(resize_transform)
#     gt_transform.append(resize_transform)
#     # normalization
#     if args.normalization:
#         std = [0.1967, 0.1896, 0.1897]
#         means = [0.3353, 0.3328, 0.2984]
#         # print("Using Normalize with mean=(0.5, 0.5, 0.5) and std=(0.5, 0.5, 0.5).")
#         image_transform.append(transforms.Normalize(mean=means, std=std))
#         # gt_transform.append(transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))) -> No normalization for groundtruth
#     # compose transforms
#     # if there is no random transforms to be applied, set it to None
#     if random_transform == []:
#         random_transform = None
#     else:
#         random_transform = transforms.Compose(random_transform)
#     image_transform = transforms.Compose(image_transform)
#     gt_transform = transforms.Compose(gt_transform)
#     # return (random_transform, image_transform, gt_transform)
#     return random_transform, image_transform, gt_transform


# def prepare_data(args):
#     # get image and groundtruth transforms (for train set)
#     random_transform, image_transform, gt_transform = prepare_transforms(args)
#     # create transforms for images and groundtruths for validation and test sets
#     # resize_transform = transforms.Pad(padding=8)
#     resize_transform = transforms.Resize((384, 384))

#     if args.normalization:
#         std = [0.1967, 0.1896, 0.1897]
#         means = [0.3353, 0.3328, 0.2984]
#         tt_transform_image = transforms.Compose(
#             [
#                 resize_transform,
#                 transforms.ToTensor(),
#                 transforms.Normalize(means, std=std),
#             ]
#         )  # TODO: Should we always normalize?
#     else:
#         tt_transform_image = transforms.Compose(
#             [resize_transform, transforms.ToTensor()]
#         )

#     tt_transform_gt = transforms.Compose(
#         [resize_transform, transforms.ToTensor()]
#     )  # No normalization for groundtruth!
#     # create base dataset
#     dataset = BaseDataset(image_folder=args.image_folder, gt_folder=args.gt_folder)
#     # seed for reproducibility
#     set_seeds()
#     # split the dataset into train, validation and test set
#     train_set, val_set, test_set = torch.utils.data.random_split(
#         dataset,
#         [
#             int(args.train_size * len(dataset)),
#             int(args.val_size * len(dataset)),
#             int(args.test_size * len(dataset)),
#         ],
#     )
#     # apply transforms
#     train_set = TransformDataset(
#         train_set,
#         random_transform=random_transform,
#         image_transform=image_transform,
#         gt_transform=gt_transform,
#     )
#     val_set = TransformDataset(
#         val_set,
#         random_transform=None,
#         image_transform=tt_transform_image,
#         gt_transform=tt_transform_gt,
#     )
#     test_set = TransformDataset(
#         test_set,
#         random_transform=None,
#         image_transform=tt_transform_image,
#         gt_transform=tt_transform_gt,
#     )
#     # create data loaders
#     train_loader = torch.utils.data.DataLoader(
#         train_set, batch_size=args.batch_size, shuffle=True
#     )
#     val_loader = torch.utils.data.DataLoader(
#         val_set, batch_size=args.batch_size, shuffle=True
#     )
#     test_loader = torch.utils.data.DataLoader(
#         test_set, batch_size=args.batch_size, shuffle=True
#     )
#     # return data loaders
#     return train_loader, val_loader, test_loader


# def prepare_model(args):
#     if args.model_name == "UNetV1":
#         print(
#             f"Initializing UNetV1 model with pretrained={args.model_pretrained}, scale={args.model_scale}."
#         )
#         model = UNetV1(pretrained=args.model_pretrained, scale=args.model_scale)
#     elif args.model_name == "UNetV2":
#         print(
#             f"Initializing UNetV2 model with in_channels={args.model_in_channels}, out_channels={args.model_out_channels}, init_features={args.model_init_features}, pretrained={args.model_pretrained}."
#         )
#         model = UNetV2(
#             in_channels=args.model_in_channels,
#             out_channels=args.model_out_channels,
#             init_features=args.model_init_features,
#             pretrained=args.model_pretrained,
#         )
#     elif args.model_name == "UNetV3":
#         print("Initializing UNetV3 model.")
#         model = UNetV3()
#     elif args.model_name == "ResNet101":
#         print("Initializing ResNet101 model.")
#         model = ResNet101()
#     ## ADD MODELS HERE!
#     return model


# def prepare_optimizer(model, args):
#     if args.optim_name == "sgd":
#         print(
#             f"Initializing SGD optimizer with lr={args.optim_lr}, momentum={args.optim_momentum}."
#         )
#         optimizer = torch.optim.SGD(
#             model.parameters(), lr=args.optim_lr, momentum=args.optim_momentum
#         )
#     elif args.optim_name == "adam":
#         print(f"Initializing Adam optimizer with lr={args.optim_lr}.")
#         optimizer = torch.optim.Adam(model.parameters(), lr=args.optim_lr)
#     return optimizer


# def step_loader(loader, n_steps=-1):
#     # creates infinite loader for training (
#     # use as metric n_steps instead of n_epochs
#     step = 0
#     while True:
#         for batch in loader:
#             yield step, batch
#             step += 1
#             if step == n_steps:
#                 return


# def calculate_metrics(preds, labels):
#     # torch.Size([2, 1, 416, 416])
#     # torch.Size([2, 1, 416, 416])

#     smooth = 1e-6
#     # convert predictions to binary format
#     preds = torch.sigmoid(preds)
#     preds = (preds > 0.5).float()

#     # pixel Accuracy
#     pixel_accuracy = (preds == labels).sum().item() / (labels.numel() + smooth)

#     # Intersection-Over-Union (IoU)
#     intersection = (preds * labels).sum()
#     union = preds.sum() + labels.sum() - intersection
#     iou = (intersection + smooth) / (union + smooth)
#     return pixel_accuracy, iou.item()


# def train(model, device, train_loader, val_loader, criterion, optimizer, args):
#     # set up WandB for logging
#     config_dict = OmegaConf.to_container(args, resolve=True)
#     wandb.init(
#         project=args.wandb_project,
#         name=args.wandb_run,
#         config=config_dict,
#         entity=args.entity,
#     )
#     # Upload the configuration file to WandB
#     wandb.config.update(config_dict)
#     best_iou_score = 0.0
#     # training loop
#     for step, batch in step_loader(train_loader, args.n_steps):
#         # training
#         model.train()
#         inputs, labels = batch
#         inputs, labels = inputs.to(device), labels.to(device)
#         optimizer.zero_grad()
#         outputs = model(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()
#         # logging
#         wandb.log({"Training Loss": loss.item()}, step=step)
#         train_pixel_accuracy, train_iou = calculate_metrics(outputs, labels)
#         wandb.log(
#             {
#                 "Training Pixel Accuracy": train_pixel_accuracy,
#                 "Training IoU": train_iou,
#             },
#             step=step,
#         )
#         # TODO
#         # validation
#         if step % args.eval_freq == 0:
#             model.eval()
#             total_val_loss = 0.0
#             total_pixel_accuracy = 0.0
#             total_iou = 0.0

#             with torch.no_grad():
#                 for val_inputs, val_targets in val_loader:
#                     val_inputs = val_inputs.to(device)
#                     val_targets = val_targets.to(device)

#                     val_outputs = model(val_inputs)
#                     val_loss = criterion(val_outputs, val_targets)
#                     total_val_loss += val_loss.item()

#                     # calculate metrics for validation data
#                     val_pixel_accuracy, val_iou = calculate_metrics(
#                         val_outputs, val_targets
#                     )
#                     total_pixel_accuracy += val_pixel_accuracy
#                     total_iou += val_iou

#             avg_pixel_accuracy = total_pixel_accuracy / len(val_loader)
#             avg_val_loss = total_val_loss / len(val_loader)
#             avg_iou = total_iou / len(val_loader)
#             wandb.log(
#                 {
#                     "Average Validation Loss": avg_val_loss,
#                     "Average Validation Pixel Accuracy": avg_pixel_accuracy,
#                     "Average Validation IoU": avg_iou,
#                 },
#                 step=step,
#             )
#             # Save the model if this is the best F1 score so far
#             if avg_iou > best_iou_score:
#                 best_iou_score = avg_iou
#                 torch.save(model.state_dict(), args.model_save_name)
#                 print("Best model saved at step: ", step)

#     return model

In [ ]:
# set seeds
set_seeds()

# create folder models if it doesn't exist
if not os.path.exists("models"):
    os.makedirs("models")

In [ ]:
args = OmegaConf.create(
    dict(
        # General
        seed=0,
        # Data folders
        image_folder="datasets/train/images/",
        gt_folder="datasets/train/groundtruth/",
        # Data tranforms
        # random_resized_crop: crop a random portion of image and resize it to a given size
        random_resized_crop=False,
        output_size=(400, 400),  # expected output size of the crop, for each edge.
        input_size=(384),  # resize
        random_resized_crop_scale=(0.5, 0.5),
        # random_horizontal_flip: randomly flip the image horizontally with a given probability
        random_horizontal_flip=True,
        # random_vertical_flip: randomly flip the image vertically with a given probability
        random_vertical_flip=True,
        # random_rotation: randomly rotate the image with a given probability
        random_rotation=False,
        degrees=5,  # range of degrees to select from
        # color_jitter: randomly change the brightness, contrast and saturation of an image
        color_jitter=False,
        brightness=0.1,  # how much to jitter brightness.
        contrast=0.1,  # how much to jitter contrast.
        saturation=0.1,  # how much to jitter saturation.
        hue=0.1,  # how much to jitter hue.
        # normalization
        normalization=True,  # TODO: Should it always be True?
        # Data loaders
        batch_size=2,
        train_size=0.8,
        val_size=0.1,
        test_size=0.1,
        # Model
        # UNetV1
        model_name="UNetV3",  # Change for v3
        model_save_name="models/checkpoints/unetv3-4.pt",
        # Optimizer
        optim_name="adam",  # sgd
        optim_lr=0.001,
        optim_momentum=0.9,  # TODO: Try with (optim_momentum != 0) and without (momentum = 0)?
        # Training
        n_steps=2000,
        eval_freq=100,
        # Wandb logging
        wandb_project="road-segmentation",
        wandb_run="unet-v3",
        entity="feeit",
    )
)

In [ ]:
# prepare train, validation and test loaders
train_loader, val_loader, test_loader = prepare_data(args)

In [ ]:
model = prepare_model(args)

In [ ]:
criterion = torch.nn.BCEWithLogitsLoss()
# criterion = DiceLoss()

In [ ]:
# prepare optimizer
optimizer = prepare_optimizer(model, args)

In [ ]:
# device to use for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
# train
trained_model = train(
    model, device, train_loader, val_loader, criterion, optimizer, args
)

In [ ]:
# from train_utils import batch_mean_and_sd

# # calculate mean and std of the dataset
# mean, std = batch_mean_and_sd(train_loader)

Submission

In [ ]:
# import transforms
from torchvision import transforms
from datasets.TestDataset import TestDataset
import matplotlib.pyplot as plt
import torch
from examples.mask_to_submission import *
import torchvision.models.segmentation as models
from models.UNetV3 import UNetV3

In [ ]:
# create segmentation model with pretrained encoder
model = UNetV3()
MODEL = "models/checkpoints/unetv3-3_effnet.pt"
checkpoint = torch.load(MODEL)
model.load_state_dict(checkpoint)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# path to the test folder
test_folder = "datasets/test/"
std = [0.1967, 0.1896, 0.1897]
means = [0.3353, 0.3328, 0.2984]

# define transformations
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize(means, std=std),
    ]
)

# create test dataset
test_dataset = TestDataset(test_folder, transform=transform)

In [ ]:
# save predictions
prediction_filenames = []
for i in range(len(test_dataset)):
    # get image
    image = test_dataset[i]
    # create prediction
    prediction = model(image.unsqueeze(0))
    # threshold prediction
    prediction = torch.sigmoid(prediction)
    prediction = (prediction > 0.5).float()
    # save prediction
    prediction_filename = "predictions/prediction_" + str(i + 1) + ".png"
    prediction_filenames.append(prediction_filename)
    plt.imsave(prediction_filename, prediction.squeeze().detach().numpy(), cmap="gray")

In [ ]:
# create submission
masks_to_submission("submission.csv", *prediction_filenames)

Post Processing

In [ ]:
import cv2
import numpy as np


def apply_morphological_ops(segmentation_map):
    # Define a kernel for operations (3x3 square)
    kernel = np.ones((3, 3), np.uint8)

    # Apply erosion
    erosion = cv2.erode(segmentation_map, kernel, iterations=4)

    # Apply dilation
    dilation = cv2.dilate(erosion, kernel, iterations=4)

    return dilation


# Assuming 'segmentation_map' is your initial segmentation result as a binary image
# processed_map = apply_morphological_ops(segmentation_map)

In [ ]:
def postprocess_segmentation(image):
    # Load the image (assuming it's a binary image with roads as white and background as black)

    # Noise reduction (using morphological opening)
    kernel_op = np.zeros((5, 5), np.uint8)
    opening = cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel_op, iterations=3)

    closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel_op, iterations=3)

    # kernel_dil = np.zeros((3, 3), np.uint8)
    # Background area determination (Dilation to increase the background area)
    # sure_bg = cv2.dilate(opening, kernel_dil, iterations=3)

    # # Identifying sure foreground area (roads)
    # dist_transform = cv2.distanceTransform(opening, cv2.DIST_L1, 5)
    # _, sure_fg = cv2.threshold(dist_transform, 0.7*dist_transform.max(), 255, 0)= 0

    return closing